In [70]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from stocks import esg_stocks2 as stocklist
import matplotlib.pyplot as plt

In [71]:
DATA_PATH = 'C://Users//aman2//Documents//BTProject//btp2.0//data//{}.csv'

def get_stock_data(stock, start_timestamp=None, end_timestamp=None):
    try:
        stock = stock.replace('.NS', '').replace('.BO', '')
        df = pd.read_csv(DATA_PATH.format(stock))

        df['Date'] = pd.to_datetime(df['Date'])
        df['Date'] = (df['Date'].astype('int64')//1e9) + 19800

        df['Date'] = pd.to_datetime(df['Date'], unit='s')
        if start_timestamp and end_timestamp:
            df = df[(df['Date'] >= start_timestamp) & (df['Date'] <= end_timestamp)]
        df = df.set_index('Date')[['Open', 'High', 'Low', 'Close', 'Volume']]
        return df
    except Exception as e:
        raise ValueError(f"Error in reading data for {stock}. Error: {e}")

def get_data(stocks, start_timestamp=None, end_timestamp=None):
    data = {}
    errors = []
    for stock in stocks:
        try:
            data[stock] = get_stock_data(stock, start_timestamp, end_timestamp)
        except Exception as e:
            errors.append(f"Error in reading data for {stock}. Error: {e}")
    return data, errors


In [72]:
# start_timestamp = '01/01/2018'
# end_timestamp = '30/6/2023'

start_timestamp = '01/01/2016'
end_timestamp = '01/01/2020'

# convert to datetime64[ns] dtype
start_timestamp = pd.to_datetime(start_timestamp, dayfirst=True)
end_timestamp = pd.to_datetime(end_timestamp, dayfirst=True)

data, errors = get_data(stocklist, start_timestamp, end_timestamp)
print('Errors: {}'.format(errors) if errors else 'No errors')
print('Stocks available: {}'.format(list(data.keys())))
stocks_available = list(data.keys())

No errors
Stocks available: ['HINDZINC', 'INDUSINDBK', 'TATACHEM', 'CUMMINSIND', 'TECHM', 'APOLLOTYRE', 'TCS', 'OFSS', 'ABB', 'INFY', 'HDFCBANK', 'SAIL', 'GAIL', 'LT', 'GODREJCP', 'TATACOMM', 'AMBUJACEM', 'JKCEMENT', 'HEROMOTOCO', 'DRREDDY', 'RELINFRA', 'EXIDEIND', 'IDFC', 'STLTECH', 'SUZLON', 'PEL', 'GLENMARK', 'SUNTV', 'BRIGADE', 'BLUESTARCO', 'NETWORK18', 'KEC', 'NEULANDLAB', 'STYRENIX', 'KIRLOSBROS', 'JUBLPHARMA', 'HIL', 'ENGINERSIN', 'INDIAGLYCO', 'EDELWEISS']


In [73]:
prices = []
for stock, df in data.items():
    prices.append(df['Close'])
prices_df = pd.concat(prices, axis=1, keys=data.keys())

In [74]:
prices_df.isna().sum().sum()

0

In [75]:
# backfill missing values
prices_df = prices_df.fillna(method='bfill')

In [76]:
prices_df.head()

,HINDZINC,INDUSINDBK,TATACHEM,CUMMINSIND,TECHM,APOLLOTYRE,TCS,OFSS,ABB,INFY,...,NETWORK18,KEC,NEULANDLAB,STYRENIX,KIRLOSBROS,JUBLPHARMA,HIL,ENGINERSIN,INDIAGLYCO,EDELWEISS
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-01,58.176147,921.572693,334.876190,873.206116,409.215424,138.269257,1031.460571,2579.260010,1058.593262,450.464447,...,59.000000,146.947784,779.306580,513.767883,162.950577,391.076691,528.682861,94.834389,93.966393,51.484661
2016-01-04,56.576931,893.223267,333.361145,860.302979,406.343353,135.443024,1011.483643,2541.708252,1062.101318,439.725037,...,58.349998,142.635452,776.610229,503.794556,156.522003,393.409821,525.536072,92.110138,93.373703,50.419456
2016-01-05,58.995258,894.227234,330.330933,852.821716,410.671173,136.747421,1002.669006,2536.992920,1037.828247,437.748352,...,57.150002,141.932373,767.834595,514.470825,153.773560,392.429901,528.165527,91.576721,95.015015,50.108768
2016-01-06,56.830467,902.545471,339.503387,848.702881,413.189087,135.095154,1016.606018,2535.500488,1021.756531,435.832764,...,57.400002,141.416763,778.522217,507.643372,151.677246,386.830444,536.959045,90.681343,95.242981,50.463840
2016-01-07,54.256138,884.904907,321.895508,844.962402,401.936798,129.442627,1012.187988,2501.510742,998.857971,428.272369,...,54.849998,134.385773,771.560669,479.864868,145.295258,387.530365,526.053528,85.575745,88.632050,48.821663


In [77]:
prices_df.isna().sum().sum()

0

In [78]:
prices_df.head()
# get all columns which have atleast one null value
null_columns = prices_df.columns[prices_df.isna().any()].tolist()

In [79]:
# make prices_df drop all the columns which have null values
prices_df = prices_df.dropna(axis=1)
prices_df.head()

,HINDZINC,INDUSINDBK,TATACHEM,CUMMINSIND,TECHM,APOLLOTYRE,TCS,OFSS,ABB,INFY,...,NETWORK18,KEC,NEULANDLAB,STYRENIX,KIRLOSBROS,JUBLPHARMA,HIL,ENGINERSIN,INDIAGLYCO,EDELWEISS
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-01,58.176147,921.572693,334.876190,873.206116,409.215424,138.269257,1031.460571,2579.260010,1058.593262,450.464447,...,59.000000,146.947784,779.306580,513.767883,162.950577,391.076691,528.682861,94.834389,93.966393,51.484661
2016-01-04,56.576931,893.223267,333.361145,860.302979,406.343353,135.443024,1011.483643,2541.708252,1062.101318,439.725037,...,58.349998,142.635452,776.610229,503.794556,156.522003,393.409821,525.536072,92.110138,93.373703,50.419456
2016-01-05,58.995258,894.227234,330.330933,852.821716,410.671173,136.747421,1002.669006,2536.992920,1037.828247,437.748352,...,57.150002,141.932373,767.834595,514.470825,153.773560,392.429901,528.165527,91.576721,95.015015,50.108768
2016-01-06,56.830467,902.545471,339.503387,848.702881,413.189087,135.095154,1016.606018,2535.500488,1021.756531,435.832764,...,57.400002,141.416763,778.522217,507.643372,151.677246,386.830444,536.959045,90.681343,95.242981,50.463840
2016-01-07,54.256138,884.904907,321.895508,844.962402,401.936798,129.442627,1012.187988,2501.510742,998.857971,428.272369,...,54.849998,134.385773,771.560669,479.864868,145.295258,387.530365,526.053528,85.575745,88.632050,48.821663


In [80]:
# log returns
returns_df = np.log(prices_df / prices_df.shift(1))
returns_df = returns_df.dropna()

In [81]:
returns_df.head()

,HINDZINC,INDUSINDBK,TATACHEM,CUMMINSIND,TECHM,APOLLOTYRE,TCS,OFSS,ABB,INFY,...,NETWORK18,KEC,NEULANDLAB,STYRENIX,KIRLOSBROS,JUBLPHARMA,HIL,ENGINERSIN,INDIAGLYCO,EDELWEISS
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,-0.027874,-0.031245,-0.004534,-0.014887,-0.007043,-0.020652,-0.019558,-0.014666,0.003308,-0.024130,...,-0.011078,-0.029785,-0.003466,-0.019603,-0.040250,0.005948,-0.005970,-0.029147,-0.006327,-0.020907
2016-01-05,0.041856,0.001123,-0.009131,-0.008734,0.010594,0.009585,-0.008753,-0.001857,-0.023119,-0.004505,...,-0.020780,-0.004941,-0.011364,0.020970,-0.017715,-0.002494,0.004991,-0.005808,0.017425,-0.006181
2016-01-06,-0.037384,0.009259,0.027389,-0.004841,0.006112,-0.012156,0.013804,-0.000588,-0.015607,-0.004386,...,0.004365,-0.003639,0.013823,-0.013360,-0.013726,-0.014371,0.016512,-0.009825,0.002396,0.007061
2016-01-07,-0.046356,-0.019739,-0.053257,-0.004417,-0.027610,-0.042742,-0.004355,-0.013496,-0.022666,-0.017499,...,-0.045442,-0.050997,-0.008982,-0.056275,-0.042987,0.001808,-0.020519,-0.057950,-0.071938,-0.033083
2016-01-08,0.012502,0.023018,0.020149,-0.002341,0.021977,0.008363,0.011051,0.005490,0.001992,0.011826,...,0.019856,-0.003494,0.001016,0.019272,-0.049961,-0.003015,0.018430,0.038219,0.013286,0.005440


In [82]:
mean_returns = returns_df.mean()
mean_returns = mean_returns
mean_returns.sort_values(ascending=False)

HDFCBANK      0.000899
BLUESTARCO    0.000872
INDIAGLYCO    0.000856
HIL           0.000823
BRIGADE       0.000788
HINDZINC      0.000759
JKCEMENT      0.000752
KEC           0.000705
EDELWEISS     0.000702
TCS           0.000681
TATACHEM      0.000597
PEL           0.000518
LT            0.000492
GODREJCP      0.000483
TECHM         0.000469
INDUSINDBK    0.000461
INFY          0.000407
EXIDEIND      0.000286
JUBLPHARMA    0.000269
STLTECH       0.000244
GAIL          0.000244
ABB           0.000169
APOLLOTYRE    0.000114
SUNTV         0.000103
HEROMOTOCO    0.000031
AMBUJACEM     0.000005
STYRENIX     -0.000033
DRREDDY      -0.000038
TATACOMM     -0.000056
SAIL         -0.000102
IDFC         -0.000114
ENGINERSIN   -0.000130
KIRLOSBROS   -0.000175
OFSS         -0.000190
CUMMINSIND   -0.000552
NEULANDLAB   -0.000647
NETWORK18    -0.000904
GLENMARK     -0.000986
SUZLON       -0.002485
RELINFRA     -0.002951
dtype: float64

In [83]:
cov_matrix = returns_df.cov()

In [84]:
cov_matrix

,HINDZINC,INDUSINDBK,TATACHEM,CUMMINSIND,TECHM,APOLLOTYRE,TCS,OFSS,ABB,INFY,...,NETWORK18,KEC,NEULANDLAB,STYRENIX,KIRLOSBROS,JUBLPHARMA,HIL,ENGINERSIN,INDIAGLYCO,EDELWEISS
HINDZINC,0.000323,0.000074,0.000085,0.000076,0.000030,0.000127,0.000009,0.000045,0.000052,0.000024,...,0.000138,0.000106,0.000051,0.000068,0.000099,0.000099,0.000099,0.000119,0.000160,0.000147
INDUSINDBK,0.000074,0.000314,0.000082,0.000070,0.000036,0.000103,0.000016,0.000041,0.000060,0.000014,...,0.000110,0.000106,0.000042,0.000045,0.000121,0.000078,0.000092,0.000107,0.000111,0.000180
TATACHEM,0.000085,0.000082,0.000262,0.000077,0.000026,0.000098,0.000016,0.000046,0.000056,0.000021,...,0.000134,0.000099,0.000057,0.000093,0.000098,0.000121,0.000117,0.000123,0.000129,0.000154
CUMMINSIND,0.000076,0.000070,0.000077,0.000324,0.000033,0.000085,0.000022,0.000045,0.000076,0.000023,...,0.000112,0.000074,0.000068,0.000061,0.000111,0.000061,0.000099,0.000103,0.000130,0.000130
TECHM,0.000030,0.000036,0.000026,0.000033,0.000297,0.000041,0.000093,0.000042,0.000027,0.000106,...,0.000032,0.000047,0.000020,0.000023,0.000023,0.000062,0.000026,0.000029,0.000065,0.000046
APOLLOTYRE,0.000127,0.000103,0.000098,0.000085,0.000041,0.000417,0.000015,0.000044,0.000067,0.000019,...,0.000176,0.000112,0.000045,0.000105,0.000149,0.000119,0.000143,0.000169,0.000159,0.000192
TCS,0.000009,0.000016,0.000016,0.000022,0.000093,0.000015,0.000211,0.000042,0.000011,0.000094,...,-0.000012,0.000016,0.000030,0.000013,0.000012,0.000021,-0.000007,-0.000002,0.000007,0.000021
OFSS,0.000045,0.000041,0.000046,0.000045,0.000042,0.000044,0.000042,0.000209,0.000025,0.000049,...,0.000051,0.000046,0.000048,0.000039,0.000049,0.000044,0.000035,0.000050,0.000055,0.000075
ABB,0.000052,0.000060,0.000056,0.000076,0.000027,0.000067,0.000011,0.000025,0.000327,0.000018,...,0.000087,0.000075,0.000036,0.000042,0.000103,0.000063,0.000101,0.000063,0.000118,0.000090
INFY,0.000024,0.000014,0.000021,0.000023,0.000106,0.000019,0.000094,0.000049,0.000018,0.000242,...,0.000014,0.000026,0.000033,0.000030,0.000017,0.000025,0.000019,0.000009,0.000009,0.000009


In [85]:
returns_df.cov()

,HINDZINC,INDUSINDBK,TATACHEM,CUMMINSIND,TECHM,APOLLOTYRE,TCS,OFSS,ABB,INFY,...,NETWORK18,KEC,NEULANDLAB,STYRENIX,KIRLOSBROS,JUBLPHARMA,HIL,ENGINERSIN,INDIAGLYCO,EDELWEISS
HINDZINC,0.000323,0.000074,0.000085,0.000076,0.000030,0.000127,0.000009,0.000045,0.000052,0.000024,...,0.000138,0.000106,0.000051,0.000068,0.000099,0.000099,0.000099,0.000119,0.000160,0.000147
INDUSINDBK,0.000074,0.000314,0.000082,0.000070,0.000036,0.000103,0.000016,0.000041,0.000060,0.000014,...,0.000110,0.000106,0.000042,0.000045,0.000121,0.000078,0.000092,0.000107,0.000111,0.000180
TATACHEM,0.000085,0.000082,0.000262,0.000077,0.000026,0.000098,0.000016,0.000046,0.000056,0.000021,...,0.000134,0.000099,0.000057,0.000093,0.000098,0.000121,0.000117,0.000123,0.000129,0.000154
CUMMINSIND,0.000076,0.000070,0.000077,0.000324,0.000033,0.000085,0.000022,0.000045,0.000076,0.000023,...,0.000112,0.000074,0.000068,0.000061,0.000111,0.000061,0.000099,0.000103,0.000130,0.000130
TECHM,0.000030,0.000036,0.000026,0.000033,0.000297,0.000041,0.000093,0.000042,0.000027,0.000106,...,0.000032,0.000047,0.000020,0.000023,0.000023,0.000062,0.000026,0.000029,0.000065,0.000046
APOLLOTYRE,0.000127,0.000103,0.000098,0.000085,0.000041,0.000417,0.000015,0.000044,0.000067,0.000019,...,0.000176,0.000112,0.000045,0.000105,0.000149,0.000119,0.000143,0.000169,0.000159,0.000192
TCS,0.000009,0.000016,0.000016,0.000022,0.000093,0.000015,0.000211,0.000042,0.000011,0.000094,...,-0.000012,0.000016,0.000030,0.000013,0.000012,0.000021,-0.000007,-0.000002,0.000007,0.000021
OFSS,0.000045,0.000041,0.000046,0.000045,0.000042,0.000044,0.000042,0.000209,0.000025,0.000049,...,0.000051,0.000046,0.000048,0.000039,0.000049,0.000044,0.000035,0.000050,0.000055,0.000075
ABB,0.000052,0.000060,0.000056,0.000076,0.000027,0.000067,0.000011,0.000025,0.000327,0.000018,...,0.000087,0.000075,0.000036,0.000042,0.000103,0.000063,0.000101,0.000063,0.000118,0.000090
INFY,0.000024,0.000014,0.000021,0.000023,0.000106,0.000019,0.000094,0.000049,0.000018,0.000242,...,0.000014,0.000026,0.000033,0.000030,0.000017,0.000025,0.000019,0.000009,0.000009,0.000009


In [86]:
len(cov_matrix)
cov_matrix.index

Index(['HINDZINC', 'INDUSINDBK', 'TATACHEM', 'CUMMINSIND', 'TECHM',
       'APOLLOTYRE', 'TCS', 'OFSS', 'ABB', 'INFY', 'HDFCBANK', 'SAIL', 'GAIL',
       'LT', 'GODREJCP', 'TATACOMM', 'AMBUJACEM', 'JKCEMENT', 'HEROMOTOCO',
       'DRREDDY', 'RELINFRA', 'EXIDEIND', 'IDFC', 'STLTECH', 'SUZLON', 'PEL',
       'GLENMARK', 'SUNTV', 'BRIGADE', 'BLUESTARCO', 'NETWORK18', 'KEC',
       'NEULANDLAB', 'STYRENIX', 'KIRLOSBROS', 'JUBLPHARMA', 'HIL',
       'ENGINERSIN', 'INDIAGLYCO', 'EDELWEISS'],
      dtype='object')

In [87]:
# Markowitz portfolio optimization
u = np.ones(len(cov_matrix))

W_mvp = np.linalg.inv(cov_matrix) @ u / (u.T @ np.linalg.inv(cov_matrix) @ u)

In [88]:
W_mvp = pd.Series(W_mvp, index=cov_matrix.index)

In [89]:
W_mvp

HINDZINC      0.064712
INDUSINDBK   -0.002534
TATACHEM      0.049584
CUMMINSIND    0.034968
TECHM         0.027521
APOLLOTYRE    0.001444
TCS           0.099083
OFSS          0.100043
ABB           0.044404
INFY          0.063102
HDFCBANK      0.304173
SAIL         -0.037573
GAIL          0.064867
LT           -0.006624
GODREJCP      0.016694
TATACOMM     -0.005584
AMBUJACEM    -0.034033
JKCEMENT      0.043784
HEROMOTOCO    0.033722
DRREDDY       0.068369
RELINFRA     -0.011897
EXIDEIND     -0.001753
IDFC         -0.017399
STLTECH      -0.003325
SUZLON       -0.001365
PEL           0.012015
GLENMARK      0.038652
SUNTV        -0.024690
BRIGADE       0.037429
BLUESTARCO    0.055425
NETWORK18    -0.005323
KEC          -0.000679
NEULANDLAB    0.014028
STYRENIX      0.025581
KIRLOSBROS    0.007614
JUBLPHARMA   -0.009817
HIL          -0.007782
ENGINERSIN    0.007780
INDIAGLYCO   -0.010288
EDELWEISS    -0.034328
dtype: float64

In [90]:
# Portfolio variance and returns

portfolio_variance = W_mvp.T @ cov_matrix @ W_mvp
portfolio_variance

4.408972095980861e-05

In [91]:
portfolio_return = mean_returns.T @ W_mvp
print(portfolio_return)
print((np.exp(portfolio_return)-1)*100)


0.0005150315275294847
0.05151641790388517


In [92]:
cov_inv = np.linalg.inv(cov_matrix)

print(np.matmul(np.transpose(mean_returns), np.linalg.solve(cov_matrix, mean_returns)))
print(mean_returns @ cov_inv @ mean_returns.T)

0.043113396259446046
0.043113396259446046


In [93]:
print(len(cov_matrix))

40


In [94]:
def portfolio_risk_for_return(mu): 
  cov_inv = np.linalg.inv(cov_matrix)
  M = np.array([[mean_returns @ np.linalg.solve(cov_matrix, mean_returns.T), u @ np.linalg.solve(cov_matrix, mean_returns.T)], [mean_returns @ np.linalg.solve(cov_matrix, u.T), u @ np.linalg.solve(cov_matrix, u.T)]])
  M_inv = np.linalg.inv(M)
  lambda12 = 2 * M_inv @ np.array([mu, 1])

  W_tan = (lambda12[0] * mean_returns @ cov_inv + lambda12[1] * u @ cov_inv)/2

  W_tan = pd.Series(W_tan, index=cov_matrix.index)

  portfolio_rrisk = W_tan.T @ cov_matrix @ W_tan
  return portfolio_rrisk, W_tan

In [99]:
def log_return_to_return(log_return):
  return np.exp(log_return*252) - 1

def sharpe_ratio(mu):
  portfolio_rrisk, W_tan = portfolio_risk_for_return(mu)
  portfolio_return = mean_returns.T @ W_tan
  return (log_return_to_return(portfolio_return) - 0.07)/np.sqrt(portfolio_rrisk*252)

In [100]:
sharpe_data = []  # List to store dictionaries of data

for mu in np.linspace(0, 0.003, 100):
    mu_return = log_return_to_return(mu)
    sharpe_ratio_value = sharpe_ratio(mu)
    risk_value = np.sqrt(portfolio_risk_for_return(mu)[0]*252)
    
    sharpe_data.append({'mu': mu_return, 'sharpe_ratio': sharpe_ratio_value, 'risk': risk_value})

# Create the DataFrame from the list of dictionaries
sharpe_df = pd.DataFrame(sharpe_data)


In [101]:
sharpe_df_sorted = sharpe_df.sort_values(by='sharpe_ratio', ascending=False)
sharpe_df_sorted.head()

,mu,sharpe_ratio,risk
99,1.129740,4.600711,0.230343
98,1.113539,4.574419,0.228125
97,1.097460,4.548039,0.225913
96,1.081504,4.521562,0.223707
95,1.065670,4.494981,0.221507


In [102]:
sharpe_df['sharpe_ratio'].corr(sharpe_df['mu'])

0.9568648173770092

In [39]:
# get the input at which portfolio_risk_for_return returns specified value using binary search
def get_input_for_output(output, input_range):
  TOL = 1e-5
  low, high = input_range
  while low < high:
    mid = (low + high) / 2
    risk, _ = portfolio_risk_for_return(mid)
    if abs(risk - output) < TOL: 
      return mid
    elif risk > output:
      high = mid
    else:
      low = mid + 0.00000001
  return -1 

x = get_input_for_output(0.008118**2, (-2, 20))
print(x)
np.exp(x*252) - 1

-1


-1.0

In [45]:
risk, _ = portfolio_risk_for_return(0.0000000000000001)
print(risk)

9.240366297748169e-05


In [26]:
0.008118**2

6.5901924e-05

In [27]:
risk, W_tan = portfolio_risk_for_return(x) 

In [28]:
np.sqrt(risk*252)

0.12883551934610318

In [29]:
W_tan

INDUSINDBK    0.003061
TATACHEM      0.104753
RBLBANK      -0.000974
ABB           0.009481
HDFCBANK      0.396014
DALBHARAT     0.004619
ADANIPOWER    0.021426
RELINFRA     -0.093032
JKCEMENT      0.073389
EXIDEIND     -0.014755
DRREDDY       0.025878
HIL           0.000120
INDIAGLYCO    0.035644
BHEL         -0.113200
NMDC          0.006509
IDFC         -0.025078
EDELWEISS    -0.026673
M&MFIN        0.027944
CSBBANK       0.292235
CGPOWER      -0.041964
AXISBANK      0.049376
HINDALCO      0.059761
ISEC         -0.015889
TATAMOTORS   -0.089352
BORORENEW     0.027518
TATASTEEL     0.012299
BAJAJHLDNG    0.056998
LTTS          0.100698
PEL           0.031848
BPCL          0.025532
MOLDTKPAC     0.006184
TATAMETALI    0.065411
GLENMARK     -0.056188
SJVN          0.007884
BLS          -0.002021
BLUESTARCO    0.081452
MAYURUNIQ    -0.062522
MOTILALOFS    0.015615
dtype: float64

In [30]:
type (W_tan)

pandas.core.series.Series

In [31]:
W_tan.index

Index(['INDUSINDBK', 'TATACHEM', 'RBLBANK', 'ABB', 'HDFCBANK', 'DALBHARAT',
       'ADANIPOWER', 'RELINFRA', 'JKCEMENT', 'EXIDEIND', 'DRREDDY', 'HIL',
       'INDIAGLYCO', 'BHEL', 'NMDC', 'IDFC', 'EDELWEISS', 'M&MFIN', 'CSBBANK',
       'CGPOWER', 'AXISBANK', 'HINDALCO', 'ISEC', 'TATAMOTORS', 'BORORENEW',
       'TATASTEEL', 'BAJAJHLDNG', 'LTTS', 'PEL', 'BPCL', 'MOLDTKPAC',
       'TATAMETALI', 'GLENMARK', 'SJVN', 'BLS', 'BLUESTARCO', 'MAYURUNIQ',
       'MOTILALOFS'],
      dtype='object')

In [32]:
start_timestamp = '01/01/2020'
end_timestamp = '30/6/2023'

# start_timestamp = '01/01/2010'
# end_timestamp = '01/01/2018'

# convert to datetime64[ns] dtype
start_timestamp = pd.to_datetime(start_timestamp, dayfirst=True)
end_timestamp = pd.to_datetime(end_timestamp, dayfirst=True)

data2, errors = get_data(W_tan.index, start_timestamp, end_timestamp)
print('Errors: {}'.format(errors) if errors else 'No errors')

prices = []
for stock, df in data2.items():
    prices.append(df['Close'])
prices_df = pd.concat(prices, axis=1, keys=data.keys())

prices_df = prices_df.fillna(method='bfill')

prices_df.isna().sum().sum()


No errors


0

In [33]:
prices_df = prices_df.dropna(axis=1)
prices_df.head()

,INDUSINDBK,TATACHEM,RBLBANK,ABB,HDFCBANK,DALBHARAT,L&TFH,ADANIPOWER,RELINFRA,JKCEMENT,...,LTTS,PEL,BPCL,MOLDTKPAC,TATAMETALI,GLENMARK,ROSSARI,SJVN,BLS,BLUESTARCO
t,,,,,,,,,,,,,,,,,,,,,
2020-01-01,1484.30,292.06,347.95,1290.00,1278.60,805.00,63.90,30.85,1176.50,186.00,...,841.57,491.65,281.61,621.65,345.70,25.75,17.20,414.45,213.55,818.80
2020-01-02,1529.00,292.69,369.55,1294.20,1286.75,820.00,64.45,32.35,1220.20,189.10,...,835.11,487.55,282.85,644.75,353.90,25.70,17.81,404.95,218.35,828.95
2020-01-03,1528.85,291.49,357.60,1322.25,1268.40,803.75,63.25,32.05,1280.05,184.10,...,834.56,483.50,283.89,641.55,352.10,25.95,18.30,408.40,209.25,841.55
2020-01-06,1469.40,287.01,336.85,1287.00,1240.95,788.95,60.10,30.45,1257.45,181.65,...,799.27,469.15,276.00,625.80,339.95,25.90,17.46,409.15,206.65,813.55
2020-01-07,1461.65,289.86,344.00,1306.75,1260.60,794.75,61.00,28.95,1283.00,182.65,...,811.04,461.00,277.54,629.40,343.20,26.10,17.50,407.23,209.35,815.50


In [34]:
returns_df = np.log(prices_df / prices_df.shift(1))
returns_df = returns_df.dropna()

In [35]:
mean_returns2 = returns_df.mean()

In [36]:
mean_returns.head()

INDUSINDBK    0.000438
TATACHEM      0.000503
RBLBANK       0.000152
ABB           0.000257
HDFCBANK      0.000866
dtype: float64

In [45]:
mean_returns2

INDUSINDBK   -0.000088
TATACHEM      0.001418
RBLBANK      -0.000746
ABB           0.001418
HDFCBANK      0.000329
DALBHARAT     0.001139
L&TFH         0.001569
ADANIPOWER    0.001726
RELINFRA      0.001215
JKCEMENT      0.000277
EXIDEIND      0.000671
DRREDDY       0.001032
HIL           0.001133
INDIAGLYCO    0.000806
BHEL          0.000166
NMDC          0.000633
IDFC         -0.000176
EDELWEISS     0.000336
M&MFIN        0.000365
CSBBANK       0.004077
CGPOWER       0.000319
AXISBANK      0.000777
HINDALCO      0.000443
ISEC          0.001349
TATAMOTORS    0.002945
BORORENEW     0.001005
TATASTEEL     0.000840
BAJAJHLDNG    0.001115
LTTS          0.000130
PEL          -0.000344
BPCL          0.001471
MOLDTKPAC     0.000284
TATAMETALI    0.000772
GLENMARK      0.000532
ROSSARI       0.002868
SJVN          0.000708
BLS           0.001023
BLUESTARCO   -0.000135
dtype: float64

In [46]:
W_tan

INDUSINDBK    0.003061
TATACHEM      0.104753
RBLBANK      -0.000974
ABB           0.009481
HDFCBANK      0.396014
DALBHARAT     0.004619
ADANIPOWER    0.021426
RELINFRA     -0.093032
JKCEMENT      0.073389
EXIDEIND     -0.014755
DRREDDY       0.025878
HIL           0.000120
INDIAGLYCO    0.035644
BHEL         -0.113200
NMDC          0.006509
IDFC         -0.025078
EDELWEISS    -0.026673
M&MFIN        0.027944
CSBBANK       0.292235
CGPOWER      -0.041964
AXISBANK      0.049376
HINDALCO      0.059761
ISEC         -0.015889
TATAMOTORS   -0.089352
BORORENEW     0.027518
TATASTEEL     0.012299
BAJAJHLDNG    0.056998
LTTS          0.100698
PEL           0.031848
BPCL          0.025532
MOLDTKPAC     0.006184
TATAMETALI    0.065411
GLENMARK     -0.056188
SJVN          0.007884
BLS          -0.002021
BLUESTARCO    0.081452
MAYURUNIQ    -0.062522
MOTILALOFS    0.015615
dtype: float64

In [38]:
W_tan.sort_values(ascending=False)

HDFCBANK      0.396014
CSBBANK       0.292235
TATACHEM      0.104753
LTTS          0.100698
BLUESTARCO    0.081452
JKCEMENT      0.073389
TATAMETALI    0.065411
HINDALCO      0.059761
BAJAJHLDNG    0.056998
AXISBANK      0.049376
INDIAGLYCO    0.035644
PEL           0.031848
M&MFIN        0.027944
BORORENEW     0.027518
DRREDDY       0.025878
BPCL          0.025532
ADANIPOWER    0.021426
MOTILALOFS    0.015615
TATASTEEL     0.012299
ABB           0.009481
SJVN          0.007884
NMDC          0.006509
MOLDTKPAC     0.006184
DALBHARAT     0.004619
INDUSINDBK    0.003061
HIL           0.000120
RBLBANK      -0.000974
BLS          -0.002021
EXIDEIND     -0.014755
ISEC         -0.015889
IDFC         -0.025078
EDELWEISS    -0.026673
CGPOWER      -0.041964
GLENMARK     -0.056188
MAYURUNIQ    -0.062522
TATAMOTORS   -0.089352
RELINFRA     -0.093032
BHEL         -0.113200
dtype: float64

In [39]:
(W_tan.index == mean_returns.index).sum()

38

In [40]:
portfolio_return = np.sum(mean_returns2 * W_tan)
print(portfolio_return*252)
print((np.exp(portfolio_return*252)-1)*100)

0.35110258276741274
42.06330509074709


In [42]:
mean_returns2.head()

INDUSINDBK   -0.000088
TATACHEM      0.001418
RBLBANK      -0.000746
ABB           0.001418
HDFCBANK      0.000329
dtype: float64

In [53]:
len(data2['ADANIPOWER'])

870